In [39]:
import requests
import pandas as pd

## <h1 style="color:orange"> 1) Obtenciòn de datos </h1> 

In [115]:
lista = ['3689da48-d557-5e5f-8347-006ced354939',
         '2a323bb8-0a6d-5bd5-8366-90041c4f1c8c',
        '2743ebad-f1e2-5eff-8c4d-f8c5191d1775',
        'a6960833-d5a3-56dc-b125-da9e4e1fce69',
        'fed214f3-332d-522c-97ac-da395a066dba']

In [114]:
dataframes = []

In [116]:
for i in lista:
    url = f'https://analytics.deacero.com/api/teenus/get-data/{i}?format=json'
    r = requests.get(url=url)
    data = r.json()
    dataframes.append(pd.DataFrame(data))
    

In [43]:
dataframes[0].head()

,ID_Pasajero,Pasajero,Edad
0,576,Danielle Thompson,60
1,579,Natalie Cuevas,49
2,683,John Murray,28
3,681,Michael Jacobs,24
4,592,Brian Hunt,40


In [44]:
dataframes[1].head()

,ID_Pasajero,Pasajero,Edad
0,596,Javier Olson,71
1,625,Monique Ramirez,35
2,637,Rob Beeghly,29
3,730,Timothy Moore,21
4,682,Scot Wooten,72


In [45]:
dataframes[2].head()

,Cve_LA,Viaje,Clase,Precio,Ruta,Cve_Cliente
0,SW,9/10/2016,Economy,60,DAL-MDW,553
1,AA,6/11/2016,Economy,150,DAL-SLC,554
2,DA,9/21/2016,Economy,68,DAL-AMA,554
3,UA,6/28/2016,Business,160,DAL-SLC,556
4,SW,6/25/2016,Economy,65,DAL-OKC,557


In [46]:
dataframes[3].head()

,Cve_LA,Viaje,Clase,Precio,Ruta,Cve_Cliente
0,SW,1/4/2017,Economy,86,DAL-ATL,637
1,SW,1/4/2017,First Class,237,DAL-ATL,402
2,SW,1/2/2017,Economy,62,DAL-LGA,191
3,SW,1/2/2017,Economy,81,DAL-MDW,191
4,AA,1/4/2017,Business,169,DAL-LGA,637


In [47]:
dataframes[4].head()

,Code,Linea_Aerea
0,AA,American Airlines
1,SW,Southwest
2,AM,Aeromexico
3,AV,Avianca
4,KL,KLM


## <h1 style="color:orange"> 2 Uniòn de las tablas </h1> 

In [174]:
pasajeros = pd.concat([dataframes[0], dataframes[1]])

In [175]:
vuelos = pd.concat([dataframes[2], dataframes[3]])

In [118]:
pasajeros.head()

,ID_Pasajero,Pasajero,Edad
0,576,Danielle Thompson,60
1,579,Natalie Cuevas,49
2,683,John Murray,28
3,681,Michael Jacobs,24
4,592,Brian Hunt,40


## Vamos a revisar la calidad de los datos:

In [176]:
def dup(df):
    dup = df.duplicated().sum()
    if(dup==0):
        print("los datos no tienen duplicados")
    else:
        print("la cantidad de duplicados es:", dup)
        if(dup/len(df)>.20):
            print("màs del 20% de los datos estàn duplicados")
            df.drop_duplicates(inplace=True)
            df.reset_index(drop=True,inplace=True)
            dup = df.duplicated().sum()
            print("la nueva cantidad de duplicados es:", dup)

In [177]:
dup(pasajeros)

la cantidad de duplicados es: 62
màs del 20% de los datos estàn duplicados
la nueva cantidad de duplicados es: 0


La cantidad de duplicados representa màs del 20% de los datos, por lo cual, personalmente verìa de nuevo la finalidad del anàlisis para decidir si aùn asì son suficientes datos o hay que recolectar màs.

In [178]:
len(pasajeros)

138

In [179]:
dup(vuelos)

los datos no tienen duplicados


In [180]:
pasajeros.head()

,ID_Pasajero,Pasajero,Edad
0,576,Danielle Thompson,60
1,579,Natalie Cuevas,49
2,683,John Murray,28
3,681,Michael Jacobs,24
4,592,Brian Hunt,40


In [181]:
pasajeros["ID_Pasajero"].duplicated().sum()

5

Tenemos 5 id repetidos, vamos a ver cuàntos registros son:

In [182]:
id_dup = list(pasajeros[pasajeros["ID_Pasajero"].duplicated()]["ID_Pasajero"])
id_dup

[582, 570, 717, 742, 562]

In [183]:
k = 0
for i in range(len(pasajeros)):
    if(pasajeros["ID_Pasajero"][i] in id_dup):
        k = k + 1


In [187]:
len(pasajeros)

128

In [185]:
for i in id_dup:
    pasajeros.drop(pasajeros.loc[pasajeros["ID_Pasajero"]==i].index, inplace = True )

In [186]:
def completitud(df):
    comple=pd.DataFrame(df.isnull().sum())
    comple.reset_index(inplace=True)
    comple=comple.rename(columns={"index":"columna",0:"total"})
    comple["completitud"]=(1-comple["total"]/df.shape[0])*100
    comple=comple.sort_values(by="completitud",ascending=True)
    comple.reset_index(drop=True,inplace=True)
    return comple

In [188]:
completitud(pasajeros)

,columna,total,completitud
0,ID_Pasajero,0,100.0
1,Pasajero,0,100.0
2,Edad,0,100.0


In [189]:
completitud(vuelos)

,columna,total,completitud
0,Cve_LA,0,100.0
1,Viaje,0,100.0
2,Clase,0,100.0
3,Precio,0,100.0
4,Ruta,0,100.0
5,Cve_Cliente,0,100.0


In [190]:
vuelos.head()

,Cve_LA,Viaje,Clase,Precio,Ruta,Cve_Cliente
0,SW,9/10/2016,Economy,60,DAL-MDW,553
1,AA,6/11/2016,Economy,150,DAL-SLC,554
2,DA,9/21/2016,Economy,68,DAL-AMA,554
3,UA,6/28/2016,Business,160,DAL-SLC,556
4,SW,6/25/2016,Economy,65,DAL-OKC,557


### Todos los vuelos estàn completos

In [191]:
vuelos.head()

,Cve_LA,Viaje,Clase,Precio,Ruta,Cve_Cliente
0,SW,9/10/2016,Economy,60,DAL-MDW,553
1,AA,6/11/2016,Economy,150,DAL-SLC,554
2,DA,9/21/2016,Economy,68,DAL-AMA,554
3,UA,6/28/2016,Business,160,DAL-SLC,556
4,SW,6/25/2016,Economy,65,DAL-OKC,557


In [192]:
len(vuelos)

400

## <h1 style="color:orange"> 3 y 4) Relacionar tablas  </h1> 

Vamos a usar la columna de Cve_Cliente y la columna de ID_Pasajero para relacionar las tablas de vuelos y pasajeros y saber què vuelo tomò cada pasajero, posteriormente vamos a unir la tabla de lìneas aereas para saber en què lìnea viajò cada pasajero, vamos a usar inner, porque eliminamos los registros con id duplicados en la tabla pasajeros, por lo tanto, el inner es para tener todos los campos completos

In [232]:
union = pd.merge(pasajeros, vuelos, left_on = "ID_Pasajero", right_on = "Cve_Cliente", how = "inner")

In [204]:
union

,ID_Pasajero,Pasajero,Edad,Cve_LA,Viaje,Clase,Precio,Ruta,Cve_Cliente
0,576,Danielle Thompson,60,AM,1/5/2016,Economy,252,DAL-SLC,576
1,576,Danielle Thompson,60,AM,11/25/2016,Business,269,DAL-TUL,576
2,576,Danielle Thompson,60,AA,9/6/2016,Business,186,DAL-AMA,576
3,576,Danielle Thompson,60,SW,4/22/2016,Economy,201,DAL-PHX,576
4,579,Natalie Cuevas,49,AV,5/13/2016,Economy,267,DAL-OKC,579
...,...,...,...,...,...,...,...,...,...
361,715,Justin Martin,48,SW,10/24/2017,First Class,204,DAL-ATL,715
362,715,Justin Martin,48,SW,11/14/2017,Economy,167,DAL-AUS,715
363,715,Justin Martin,48,AM,11/16/2017,First Class,81,DAL-ATL,715
364,715,Justin Martin,48,SW,11/24/2017,First Class,210,DAL-OKC,715


In [205]:
completitud(union)

,columna,total,completitud
0,ID_Pasajero,0,100.0
1,Pasajero,0,100.0
2,Edad,0,100.0
3,Cve_LA,0,100.0
4,Viaje,0,100.0
5,Clase,0,100.0
6,Precio,0,100.0
7,Ruta,0,100.0
8,Cve_Cliente,0,100.0


como eliminamos de la tabla de pasajeros los id repetidos, vamos a eliminar de la tabla vuelos, los registros que tengan esos id

In [199]:
len(union)

366

Ahora vamos a unir la tabla anterior (union) con la tabla de lineas aereas usando la columna de "Cva_LA" y Code mediante un merge  con inner para que nos traiga los elementos que encuentre en ambas tablas

In [207]:
dataframes[4]

,Code,Linea_Aerea
0,AA,American Airlines
1,SW,Southwest
2,AM,Aeromexico
3,AV,Avianca
4,KL,KLM


In [233]:
union = pd.merge(union, dataframes[4], left_on = "Cve_LA", right_on = "Code", how = "inner")

In [215]:
union.head(2)

,ID_Pasajero,Pasajero,Edad,Cve_LA,Viaje,Clase,Precio,Ruta,Cve_Cliente,Code,Linea_Aerea
0,576,Danielle Thompson,60,AM,1/5/2016,Economy,252,DAL-SLC,576,AM,Aeromexico
1,576,Danielle Thompson,60,AM,11/25/2016,Business,269,DAL-TUL,576,AM,Aeromexico


como la cantidad de registros disminuyò, podemos decir que o el catàlogo no està completo o que la clave en la tabla de vuelos està mal escrita

# Columnas importantes:

Hay que seleccionar del df union las columnas que nos interesan y lo vamos a guardar en el mismo df union

In [234]:
union = union[["Viaje", "Clase","Precio", "Ruta", "Edad", "Linea_Aerea"]]
union


,Viaje,Clase,Precio,Ruta,Edad,Linea_Aerea
0,1/5/2016,Economy,252,DAL-SLC,60,Aeromexico
1,11/25/2016,Business,269,DAL-TUL,60,Aeromexico
2,1/27/2016,First Class,190,DAL-OKC,28,Aeromexico
3,11/13/2016,Business,142,DAL-OKC,40,Aeromexico
4,11/3/2016,Business,56,DAL-TUL,40,Aeromexico
...,...,...,...,...,...,...
346,7/24/2017,First Class,214,DAL-ATL,46,KLM
347,8/2/2017,Business,121,DAL-AMA,46,KLM
348,8/11/2017,First Class,204,DAL-AMA,46,KLM
349,3/14/2017,Economy,73,DAL-PHX,55,KLM


## <h1 style="color:orange"> 5) Promedio  </h1> 

Primero con la columna Viaje, que es la fecha del vuelo, vamos a obtener el año y el semestre:

Vamos a revisar el tipo de dato de la columna Viaje:

In [235]:
union["Viaje"].dtype

dtype('O')

In [236]:
union.head()

,Viaje,Clase,Precio,Ruta,Edad,Linea_Aerea
0,1/5/2016,Economy,252,DAL-SLC,60,Aeromexico
1,11/25/2016,Business,269,DAL-TUL,60,Aeromexico
2,1/27/2016,First Class,190,DAL-OKC,28,Aeromexico
3,11/13/2016,Business,142,DAL-OKC,40,Aeromexico
4,11/3/2016,Business,56,DAL-TUL,40,Aeromexico


In [238]:
union["Viaje"]=pd.to_datetime(union["Viaje"],format="%m/%d/%Y")

In [239]:
union["Viaje"].dtype

dtype('<M8[ns]')

In [263]:
union["semestre"]=union["Viaje"].map(lambda x: x.month)
union["year"]=union["Viaje"].map(lambda x: x.year)

In [264]:
union["semestre"]=union["semestre"].map( lambda x: 1 if x<=6 else 2)

In [265]:
union.head()

,Viaje,Clase,Precio,Ruta,Edad,Linea_Aerea,semestre,year
0,2016-01-05,Economy,252,DAL-SLC,60,Aeromexico,1,2016
1,2016-11-25,Business,269,DAL-TUL,60,Aeromexico,2,2016
2,2016-01-27,First Class,190,DAL-OKC,28,Aeromexico,1,2016
3,2016-11-13,Business,142,DAL-OKC,40,Aeromexico,2,2016
4,2016-11-03,Business,56,DAL-TUL,40,Aeromexico,2,2016


In [267]:
union = union[["Clase","Precio", "Ruta", "Linea_Aerea", "semestre", "year"]]

In [269]:
union = union.groupby(["Clase", "Ruta", "Linea_Aerea", "semestre", "year"], as_index = False).mean()